In [2]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import seaborn as sns

In [4]:
!pip3 install gensim
!pip3 install pyLDAvis
# !conda init bash


In [5]:
from gensim import corpora, models
from gensim.similarities import MatrixSimilarity
from gensim.utils import SaveLoad
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from re import sub
import pyLDAvis
from collections import Counter
from gensim.matutils import corpus2csc, sparse2full, corpus2dense
from wordcloud import WordCloud
from sklearn.utils import resample

In [6]:
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)
from src.loader import SlackDataLoader
from src.preprocessing import Preprocessor
slack_data = SlackDataLoader('../anonymized/')
data = slack_data.slack_parser('../anonymized/*/')

[nltk_data] Downloading package wordnet to /home/basilel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/basilel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
preprocess = Preprocessor(data)

In [8]:
data.head()

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel
0,message,<!here> we're on gmeet...if you're online join us,Judith Bolton,1663139588.116659,broadcast,0,0,0,0,0,
1,message,<@U03UH397319> <https://anderfernandez.com/en/...,Judith Bolton,1663142046.799109,user,0,0,0,0,0,
2,message,when running your API try `python3 -m uvicorn ...,Judith Bolton,1663142094.022539,text,0,0,0,0,0,
3,message,Well received with thanks <@U03UJGP0C68>,Brenda Hernandez,1663142486.253059,text,0,0,0,0,0,
4,message,Good afternoon everyone. I am sorry for having...,Brian Odom,1663151073.400859,text,1663151073.400859,2,2,"U03UH397319,U03UJKJGRAQ",1663154349.239879,


In [9]:
data.shape

(17748, 11)

In [10]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [11]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data = preprocess.clean_text(org_col='msg_content',new_col='clean_msg_content')
data = preprocess.stem('clean_msg_content')

In [12]:
data['msg_content'][2]

'when running your API try `python3 -m uvicorn main:app --reload` or `python -m uvicorn main:app --reload`'

In [13]:
data['clean_msg_content'][2]

'run api tri `python3 -m uvicorn main:app --reload` `python -m uvicorn main:app --reload`'

In [14]:
data.to_csv('../clean_messages.csv')

In [15]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/basilel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/basilel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
# Used stemmer instead of lemmatizer
cleanMessageList = preprocess.filterMessageList(data['clean_msg_content'])

In [17]:
cleanMessageList[:5]

[['<here>', 'gmeetif', 'onlin', 'join'],
 ['<uuh>', '<httpsanderfernandezcomenbloghow-to-create-api-python>'],
 ['run',
  'api',
  'tri',
  '`python',
  '-m',
  'uvicorn',
  'mainapp',
  '--reload`',
  '`python',
  '-m',
  'uvicorn',
  'mainapp',
  '--reload`'],
 ['well', 'receiv', 'thank', '<uujgpc>'],
 ['good',
  'afternoon',
  'everyone',
  'sorri',
  'miss',
  'meet',
  'earlier',
  'feel',
  'well',
  'today']]

In [18]:
def makeDict(myTweetList):
    """Create dictionary from list of tokenized documents"""
    return corpora.Dictionary(myTweetList)

def makeCorpus(myTweetList,myDict):
    """Create corpus from list of tokenized documents"""
    return [myDict.doc2bow(tweet) for tweet in myTweetList]

def createLDA(myCorpus, myDictionary,myTopics=10,myPasses=10,myIterations=50,myAlpha=0.001):
    """LDA model call function"""
    return models.LdaMulticore(myCorpus, id2word=myDictionary, num_topics=myTopics, passes=myPasses,
    iterations=myIterations,alpha=myAlpha)

In [19]:
"""Create model objects"""
kagDict   = makeDict(cleanMessageList)
kagCorpus = makeCorpus(cleanMessageList, kagDict)
kagLda = createLDA(kagCorpus, kagDict)

"""Save model objects"""
SaveLoad.save(kagLda,'kaggleLDAmodel')
corpora.MmCorpus.serialize('kaggleCorpus.mm', kagCorpus)
kagDict.save('kaggleDictionary.dict')

In [20]:
kagLda = SaveLoad.load('kaggleLDAmodel')
kagDict = corpora.Dictionary.load('kaggleDictionary.dict')
kagCorpus = corpora.MmCorpus('kaggleCorpus.mm')

In [21]:
!pip3 install pyLDAvis

In [22]:
# import pyLDAvis
import pyLDAvis.gensim_models

In [23]:
pyLDAvis.enable_notebook()
ldaViz = pyLDAvis.gensim_models.prepare(kagLda, kagCorpus, kagDict)

In [24]:
kagLda.print_topics()

[(0,
  '0.040*"file" + 0.029*"=" + 0.018*"error" + 0.017*"yeah" + 0.014*"line" + 0.009*"get" + 0.008*"permiss" + 0.006*"return" + 0.006*"command" + 0.006*"deploy"'),
 (1,
  '0.011*"good" + 0.011*"make" + 0.010*"link" + 0.010*"guy" + 0.009*"like" + 0.008*"today" + 0.008*"meet" + 0.008*"share" + 0.008*"present" + 0.007*"let"'),
 (2,
  '0.044*"session" + 0.034*"plea" + 0.033*"time" + 0.030*"-" + 0.030*"<here>" + 0.028*"next" + 0.026*"pm" + 0.026*"min" + 0.023*"utc" + 0.020*"meet"'),
 (3,
  '0.046*"hello" + 0.016*"<uuvhcvkb>" + 0.016*"get" + 0.015*"<uvamtfa>" + 0.015*"<uujgpc>" + 0.008*"got" + 0.008*"thank" + 0.007*"love" + 0.006*"<uuura>" + 0.006*"point"'),
 (4,
  '0.022*"group" + 0.018*"know" + 0.017*"sure" + 0.013*"let" + 0.009*"week" + 0.008*"one" + 0.008*"happi" + 0.007*"list" + 0.006*"find" + 0.006*"number"'),
 (5,
  '0.029*"data" + 0.011*"think" + 0.010*"connect" + 0.010*"run" + 0.010*"creat" + 0.009*"tri" + 0.008*"one" + 0.008*"airflow" + 0.008*"like" + 0.008*"docker"'),
 (6,
  '0.

In [25]:
ldaViz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.331401  0.103408       1        1  13.880707
5      0.104229  0.048091       2        1  12.894826
9      0.027926 -0.086825       3        1  11.420449
7     -0.043570 -0.071115       4        1  11.276941
1     -0.076132 -0.091414       5        1   9.979305
8      0.067426  0.141733       6        1   9.540294
6      0.051964 -0.034625       7        1   8.863862
4      0.014999 -0.077356       8        1   8.597093
3      0.063377 -0.117190       9        1   7.414999
0      0.121181  0.185292      10        1   6.131523, topic_info=         Term        Freq       Total Category  logprob  loglift
16      thank  654.000000  654.000000  Default  30.0000  30.0000
57       work  830.000000  830.000000  Default  29.0000  29.0000
306   session  561.000000  561.000000  Default  28.0000  28.0000
40      hello  352.000000  352.000000  Default  27.0000  27.0000
177        ye  316.000000  316.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
263    instal   28.255018  235.512694  Topic10  -5.2907   0.6712
1049    found   16.796813   62.847528  Topic10  -5.8108   1.4722
12        tri   19.381364  500.880978  Topic10  -5.6677  -0.4603
269      want   16.907032  258.209104  Topic10  -5.8042   0.0657
605      last   15.879308  116.769186  Topic10  -5.8670   0.7965

[701 rows x 6 columns], token_table=       Topic      Freq                                               Term
term                                                                     
550       10  0.888250  "homejdsdocumentsdvcessyvenvlibpythonsite-pack...
554       10  0.888247                         "usrlibpythoncontextlibpy"
354        3  0.090797                                                  #
354        5  0.030266                                                  #
354       10  0.847435                                                  #
...      ...       ...                                                ...
8430       4  0.937141                                            yididya
8158       9  0.910123                                               yolo
3097       6  0.929600                                            youtube
3851       9  0.933193                                                yup
11116      4  0.913925                                                  |

[1930 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 6, 10, 8, 2, 9, 7, 5, 4, 1])

In [26]:
def translateLdaIdx(myLdaModel, myLdaViz):
    """Translate lda model topics to match the topics in pyLDAvis visualization"""
    ldaVizIdx = myLdaViz[0].index
    return list(ldaVizIdx)

In [27]:
newIdx = translateLdaIdx(kagLda,ldaViz)

In [28]:
def createDenseMat(myLdaModel,myCorpus,newIdx):
    """Transform corpus to dataframe with topics matching lda visualization"""
    numTopics = myLdaModel.num_topics
    myDense = corpus2dense(myLdaModel[myCorpus],numTopics)
    myDf = pd.DataFrame(myDense)
    mySortedDf = myDf.transpose()
    mySortedDf = myDf.transpose()[newIdx]
    mySortedDf.columns = ['topic' + str(i + 1) for i in range(numTopics)]
    return mySortedDf

In [29]:
kagDf = createDenseMat(kagLda,kagCorpus,newIdx)

In [30]:
import numpy as np

def sortByTopicToIdx(cleanedTweetList, mySortedDf, myTopic, myTopicThresh=0.1):
    """Returns an index of tweets surpassing a topic value threshold"""
    myCleanArray = np.array(cleanedTweetList, dtype=object)
    srtIdx = list(mySortedDf[mySortedDf[myTopic] > myTopicThresh].index)
    return srtIdx

# Example usage
sortedIdx = sortByTopicToIdx(cleanMessageList, kagDf, 'topic2', myTopicThresh=0.1)


def sortTweetsByIdx(cleanedTweetList,srtIdx):
    """Returns sorted tweets as a list based on a defined sort index"""
    myCleanArray = np.array(cleanedTweetList, dtype=object)
    srtTweets = list(myCleanArray[srtIdx])
    return srtTweets

In [31]:
!pip3 install numpy


In [32]:
sortedIdx = sortByTopicToIdx(cleanMessageList,kagDf,'topic2',myTopicThresh=0.1)

In [33]:
# Word cloud for topic 
def makeWordCloud(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1):
    """Create word cloud of tweets passing a given threshold for a given topic"""
    sortedIdx = sortByTopicToIdx(cleanedTweetList,mySortedDf,myTopic,myTopicThresh=0.1)
    mySortedTweets = sortTweetsByIdx(cleanedTweetList,sortedIdx)
    filteredWords = ' '.join([' '.join(string) for string in mySortedTweets])
    myTopicCloud = WordCloud(max_font_size=100,scale=8).generate(filteredWords)
    fig = plt.figure(figsize=(10,10), dpi=1600)
    plt.imshow(myTopicCloud)
    plt.axis("off")
    plt.show()

In [34]:
# Example usage
makeWordCloud(cleanMessageList,kagDf,'topic2',myTopicThresh=0.2)

/home/basilel/anaconda3/lib/python3.11/site-packages/wordcloud/wordcloud.py:106: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  self.colormap = plt.cm.get_cmap(colormap)
